In [1]:
import numpy as np
import itertools

In [61]:
def makeSymbolList(numSymbolOnReel, numSymbolTotal, precision):
    fenceposts = [round(i, int(np.log10(precision))) for i in list(np.random.random(size=numSymbolOnReel - 1))]
    fenceposts = sorted([0] + fenceposts + [1])
    symbols = np.random.choice(numSymbolTotal, size=numSymbolOnReel, replace=False)
    symbolList = [(symbols[i],round((fenceposts[i+1] - fenceposts[i]), 
                                    int(np.log10(precision)))) for i in range(numSymbolOnReel)]
    return symbolList
    
def makeMachine(numSymbolList, numSymbolsTotal, precision):
    if numSymbolsTotal < max(numSymbolList):
        print("Total number of symbols too small...")
        return
    numReel = len(numSymbolList)
    machineDefList = [sorted(makeSymbolList(numSymbolList[i], 
                                            numSymbolsTotal, precision),
                             key=lambda x: x[1], reverse = True) for i in range(numReel)]
    print("Probabilities sum to 1? " + str([sum([j[1] for j in i]) for i in machineDefList]))
    return machineDefList

def evaluateMachine(machineDefList):
    numReel = len(machineDefList)
    probDict = {i:0 for i in range(1, numReel + 1)}
    for thisTuple in itertools.product(*machineDefList):
        thisCount = numReel + 1 - len(set([i[0] for i in thisTuple]))
        thisProb = np.product([i[1] for i in thisTuple])
        if thisCount == numReel:
            probDict[str(thisTuple[0][0]) * numReel] = thisProb
        probDict[thisCount] += thisProb
    return probDict

def probDictToSolidity(probDict, payoutKeys):
    payoutString = ""
    numPayout = len(payoutKeys)
    for thisKey in payoutKeys:
        thisProb = probDict[thisKey]
        thisString = "    if (maxMatch == %s) {\n      return betAmount * %s;\n    }\n" % (thisKey, str(int(1./(thisProb * numPayout))))
        payoutString += thisString
    return payoutString

def makeMachineToSolidity(machine):
    flatMachine = [item for sublist in machine for item in sublist]
    precision = max([10**(len(str(i[1]).split(".")[1])) for i in flatMachine])
    solidityString = "function makeMachine() internal {\n"
    for index, reel in enumerate(machine):
        reelString0 = "    reels[%s].probDenominator = %s;\n" % (str(index), str(precision))
        reelString1 = "    reels[%s].probs = %s;\n" % (str(index), str([int(i[1] * precision) for i in reel]))
        reelString2 = "    reels[%s].eventLabels = %s;\n" % (str(index), str([i[0] for i in reel]))
        
        reelString = reelString0 + reelString1 + reelString2
        solidityString += reelString
    return solidityString + "}"

def makeSimpleMachine(numReels, numSymbols, probList):
    if sum(probList) != 1:
        print("Invalid probability list...")
        return None
    else:
        return [[(j,probList[j]) for j in range(numSymbols)] for i in range(numReels)]

In [62]:
numSymbolOnReel = [8,8,8,8,8]
numSymbolTotal = 8
precision = 1000

numReel = len(numSymbolOnReel)

#machineOne = makeMachine(numSymbolOnReel, numSymbolTotal, precision)
machineOne = makeSimpleMachine(5, 8, [.12, .12, .12, .12, .13, .13, .13, .13])

In [63]:
probDict = evaluateMachine(machineOne)
print(probDict)
print(sum([i[1] for i in probDict.items() if type(i[0]) == int]))

{1: 0.2046096000000028, 2: 0.5124605999999942, 3: 0.2568866999999949, 4: 0.025795050000000135, 5: 0.00024805, '00000': 2.48832e-05, '11111': 2.48832e-05, '22222': 2.48832e-05, '33333': 2.48832e-05, '44444': 3.712930000000001e-05, '55555': 3.712930000000001e-05, '66666': 3.712930000000001e-05, '77777': 3.712930000000001e-05}
0.9999999999999921


In [64]:
payoutString = probDictToSolidity(probDict,[4,5])

In [65]:
makeMachineString = makeMachineToSolidity(machineOne)

In [66]:
solidityCode = '''
// ******************************
// DEFINE THE MACHINE

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = %s;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  %s
  
  // CALCULATE THE PAYOUT
  function paytable(uint[numReel] memory outcome, uint betAmount) internal returns (uint) {
    uint maxMatch = countMaxMatch(outcome);
%s}
  
// END MACHINE DEFINITION
// ***************************************
  
''' % (str(numReel), makeMachineString, payoutString)

In [67]:
print(solidityCode)


// ******************************
// DEFINE THE MACHINE

  // MACHINE SPECIFIC CONSTANTS AND VARIABLES
  uint8 constant numReel = 5;
  
  // DEFINE THE PROBABILITIES FOR THE REEL SYMBOLS
  function makeMachine() internal {
    reels[0].probDenominator = 100;
    reels[0].probs = [12, 12, 12, 12, 13, 13, 13, 13];
    reels[0].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7];
    reels[1].probDenominator = 100;
    reels[1].probs = [12, 12, 12, 12, 13, 13, 13, 13];
    reels[1].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7];
    reels[2].probDenominator = 100;
    reels[2].probs = [12, 12, 12, 12, 13, 13, 13, 13];
    reels[2].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7];
    reels[3].probDenominator = 100;
    reels[3].probs = [12, 12, 12, 12, 13, 13, 13, 13];
    reels[3].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7];
    reels[4].probDenominator = 100;
    reels[4].probs = [12, 12, 12, 12, 13, 13, 13, 13];
    reels[4].eventLabels = [0, 1, 2, 3, 4, 5, 6, 7];
}
  
  // CALCULATE THE PAYOUT
  function paytable(uint[